# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import pickle
import requests
from io import BytesIO

from dataclasses import dataclass
@dataclass(frozen=True)
class Url:
    """Wrapper around a URL string to provide nice display in IPython environments."""
    # from: https://github.com/jupyterlab/jupyterlab/issues/7393

    __url: str

    def _repr_html_(self):
        """HTML link to this URL."""
        return f'<a href="{self.__url}">{self.__url}</a>'

    def __str__(self):
        """Return the underlying string."""
        return self.__url

# Initialize

In [2]:
def load_cards(path='./cards.pkl'):

    with open(path, 'rb') as f:
        return pickle.load(f)
        
study_questions = load_cards()
question_log = [0]

In [3]:
def new_question(topic, question, answers, images, videos=[]):
    """Question and answers for new flash card
    topic: text containing the topic
    question: text contianing the question
    answers: list containing answers to question
    images: list containing n number of image paths pertaining to question
    videos: list containing n number of links to web videos for question"""
    global study_questions
    
    get_qid = 1
    if topic in study_questions.keys():
        get_qid = len(study_questions[topic].keys()) + 1
    else:
        study_questions[topic] = {}

    answer = {
        'text': answers,
        'img': images,
        'video': videos
    }
    study_questions[topic][get_qid] = {'question': question,
                                        'answer': answer}


In [4]:
def save_cards(cards=study_questions, path='./cards.pkl'):
    
    with open(path, 'wb') as f:
        pickle.dump(study_questions, f)

In [5]:
def view_flashcard(q_id=0, topic='data_science'):
    """Select a flash card by id number. If q_id is 0, a random question will be displayed"""

    global question_log
    rng = 1

    if q_id != 0:
        try:
            # print question
            print(study_questions[topic][q_id]['question'])
            # update question log
            question_log.append(q_id)
        except:
            max_id = (len(study_questions[topic].keys()) + 1)
            print(f'Invalid question ID. Try a number between 1-{max_id}')

    else:
        # get random question id
        rng = np.random.choice(
                range(
                    1, (len(study_questions[topic].keys()) + 1) 
                    )
                )
        q_id = rng
        
        # update question log
        question_log.append(q_id)
        
        # print question
        print(study_questions[topic][q_id]['question'])

In [6]:
def view_answer(q_id=0, topic='data_science'):
    if q_id == 0:
        q_id = question_log[-1]
    text_answers = study_questions[topic][q_id]['answer']['text']
    for answer in text_answers:
        print(answer)
    
    image_answers = study_questions[topic][q_id]['answer']['img']
    if len(image_answers) > 0:
        for image in image_answers:
            response = requests.get(image)
            im = Image.open(BytesIO(response.content))
            display(im) 
    
    video_answers = study_questions[topic][q_id]['answer']['video']
    if len(video_answers) > 0:
        for video in video_answers:
            display(Url(video))    

In [7]:
def reset_log():
    global question_log
    question_log = [0]
    print('Question log reset.')

# Make New Flashcard

In [ ]:
topic = 'data_science'
question = "What is entropy and how is it calculated"
answers = ["In information theory, the entropy of a random variable is the average level of 'information', 'surprise', or 'uncertainty' inherent to the variable's possible outcomes.",
           "[0, 0, 0, 1, 1, 1, 1, 1]",
           "Putting p=5 and n=8, we get",
           "Entropy = -(5/8 log(5/8) + 3/8 log(3/8))"]
images = []
videos = ["https://www.youtube.com/watch?v=YtebGVx-Fxw"
          ]

In [ ]:
new_question(topic, question, answers, images, videos)

In [ ]:
save_cards(study_questions)

# Flashcard Tools

In [8]:
reset_log()

Question log reset.


In [9]:
view_flashcard(7)

You are given a data set consisting of variables with more than 30 percent missing values. How will you deal with them?


In [10]:
question_log

[0, 7]

In [11]:
view_flashcard()

What are recommender systems?


In [12]:
view_answer()

A recommender system predicts what a user would rate a specific product based on their preferences. It can be split into two different areas:
Collaborative Filtering: ex: Last.fm recommends tracks that other users with similar interests play often. This is also commonly seen on Amazon after making a purchase; customers may notice the following message accompanied by product recommendations: 'Users who bought this also bought…'
Content-based Filtering: ex: Pandora uses the properties of a song to recommend music with similar properties. Here, we look at content, instead of looking at who else is listening to music.


Url(_Url__url='https://www.youtube.com/watch?v=U-yq3I9QugQ')

# Export New Cards

In [ ]:
save_cards(study_questions)